# Data Cleaning - USA companies
Clean the dataset by handling missing values, removing duplicates, correcting data types, and standardizing formats. This step ensures the data is ready for analysis and modeling.

In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
import os

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [11]:
pd.read_csv('usa_companies.csv').head()

,id,Unnamed: 0.1,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,short_description,description,overview,tag_list,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,lat,lng,ROI
0,c:1,0,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,NaN,wetpaint-inc.com,http://wetpaint-inc.com,BachelrWetpaint,http://s3.amazonaws.com/crunchbase_prod_assets...,401.00,54.00,NaN,Technology Platform Company,Wetpaint is a technology platform company that...,"wiki, seattle, elowitz, media-industry, media-...",USA,WA,Seattle,Seattle,NaN,NaN,NaN,NaN,2005-10-01,2008-05-19,3.00,39750000.00,2010-09-05,2013-09-18,5.00,17.00,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,47.61,-122.33,15.50
1,c:10,1,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,NaN,NaN,flektor.com,http://www.flektor.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,186.00,85.00,NaN,NaN,Flektor is a rich-media mash-up platform that ...,"flektor, photo, video",USA,CA,Culver City,Los Angeles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,34.02,-118.40,NaN
2,c:100,2,Company,100,NaN,There,there,/company/there,games_video,acquired,NaN,NaN,there.com,http://www.there.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,107.00,34.00,NaN,NaN,There.com is an online virtual world where any...,"virtualworld, there, teens",USA,CA,San Mateo,SF Bay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003-02-01,2011-09-23,4.00,12.00,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,37.56,-122.33,NaN
3,c:10005,8,Company,10005,NaN,Thomas Publishing,thomas publishing,/company/thomas-publishing,advertising,operating,NaN,NaN,thomaspublishing.com,http://www.thomaspublishing.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,276.00,47.00,NaN,Online Media,"For more than a century, Thomas Publishing has...",NaN,USA,NY,New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999-06-01,1999-06-01,1.00,2.00,NaN,2008-08-24 20:21:21,2009-11-19 17:21:00,40.71,-74.01,NaN
4,c:10009,12,Company,10009,NaN,dimension5 labs,dimension5 labs,/company/dimension5-labs,advertising,operating,2008-08-01,NaN,d5labs.com,http://d5labs.com,NaN,http://s3.amazonaws.com/crunchbase_prod_assets...,300.00,85.00,NaN,NaN,dimension5 labs is a full service digital adve...,"virtual-worlds, advertising-agency, complex-pr...",USA,NM,Santa Fe,Santa Fe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-08-22,2008-08-22,1.00,2.00,NaN,2008-08-24 21:54:55,2008-12-21 17:21:53,35.69,-105.94,NaN


In [12]:
companies_df = pd.read_csv('/Users/aminosaurier/Downloads/spring_2025_startup_survival/usa_companies.csv', sep=',', header=0)

# Check for missing values
missing_values = companies_df.isnull().sum()
missing_percentage = (missing_values / len(companies_df)) * 100

# Create a DataFrame to display missing values information
missing_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing Percentage': missing_percentage
})

print("\nMissing values analysis:")
print(missing_info[missing_info['Missing Values'] > 0].sort_values('Missing Percentage', ascending=False))



Missing values analysis:
                     Missing Values  Missing Percentage
parent_id                     51637              100.00
ROI                           51057               98.88
last_investment_at            50886               98.55
first_investment_at           50886               98.55
invested_companies            50884               98.54
investment_rounds             50884               98.54
closed_at                     50258               97.33
short_description             46808               90.65
funding_total_usd             33495               64.87
first_funding_at              32004               61.98
last_funding_at               32004               61.98
funding_rounds                31905               61.79
last_milestone_at             24967               48.35
first_milestone_at            24967               48.35
milestones                    24967               48.35
tag_list                      22782               44.12
twitter_username      

In [13]:
# Identify columns to drop based on high missing percentage or redundancy
print("\nIdentifying columns to drop...")

# Columns with very high missing values (>95%) that aren't critical
high_missing_cols = missing_info[missing_info['Missing Percentage'] > 95].index.tolist()
print(f"Columns with >95% missing values: {high_missing_cols}")

# Redundant identifier columns
redundant_cols = ['Unnamed: 0.1']  # This appears to be just a row index
print(f"Redundant columns: {redundant_cols}")

# Columns with limited analytical value and categorical cols
limited_value_cols = ['entity_id', 'id', 'created_by', 'lat', 'domain', 'twitter_username', 'homepage_url', 'permalink', 'normalized_name', 'lng', 'tag_list', 'overview', 'description', 'short_description', 'logo_url', 'logo_width', 'logo_height', 'parent_id']
print(f"Columns with limited analytical value: {limited_value_cols}")

# Combine all columns to drop
columns_to_drop = high_missing_cols + redundant_cols + limited_value_cols
print(f"\nTotal columns to drop: {len(columns_to_drop)}")
print(columns_to_drop)

# Drop the identified columns
df = pd.read_csv('/Users/aminosaurier/Downloads/spring_2025_startup_survival/usa_companies.csv', sep=',', header=0)
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')
print(f"\nShape after dropping columns: {df_cleaned.shape}")


Identifying columns to drop...
Columns with >95% missing values: ['parent_id', 'closed_at', 'first_investment_at', 'last_investment_at', 'investment_rounds', 'invested_companies', 'ROI']
Redundant columns: ['Unnamed: 0.1']
Columns with limited analytical value: ['entity_id', 'id', 'created_by', 'lat', 'domain', 'twitter_username', 'homepage_url', 'permalink', 'normalized_name', 'lng', 'tag_list', 'overview', 'description', 'short_description', 'logo_url', 'logo_width', 'logo_height', 'parent_id']

Total columns to drop: 26
['parent_id', 'closed_at', 'first_investment_at', 'last_investment_at', 'investment_rounds', 'invested_companies', 'ROI', 'Unnamed: 0.1', 'entity_id', 'id', 'created_by', 'lat', 'domain', 'twitter_username', 'homepage_url', 'permalink', 'normalized_name', 'lng', 'tag_list', 'overview', 'description', 'short_description', 'logo_url', 'logo_width', 'logo_height', 'parent_id']

Shape after dropping columns: (51637, 19)


In [14]:
# Check for outliers in numeric columns
print("\nChecking for outliers in numeric columns...")

# Function to detect outliers using IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return outliers.count()

# Check outliers in key numeric columns
numeric_cols_for_outliers = ['funding_total_usd', 'funding_rounds', 'relationships', 'age_years']
for col in numeric_cols_for_outliers:
    if col in df_cleaned.columns and df_cleaned[col].notnull().sum() > 0:
        outlier_count = detect_outliers(df_cleaned, col)
        print(f"Found {outlier_count} outliers in {col}")

# For funding_total_usd, cap extreme values at 99.9 percentile
if 'funding_total_usd' in df_cleaned.columns:
    cap_value = df_cleaned['funding_total_usd'].quantile(0.999)
    extreme_values = df_cleaned['funding_total_usd'] > cap_value
    if extreme_values.sum() > 0:
        print(f"Capping {extreme_values.sum()} extreme funding values at {cap_value}")
        df_cleaned.loc[extreme_values, 'funding_total_usd'] = cap_value


Checking for outliers in numeric columns...
Found 2137 outliers in funding_total_usd
Found 2024 outliers in funding_rounds
Found 3273 outliers in relationships
Capping 19 extreme funding values at 833952250.0000048


In [15]:
# Save the cleaned dataset
print("\nSaving cleaned dataset...")
df_cleaned.to_csv('usa_cleaned_companies.csv', index=False)
print("Saved cleaned dataset to 'usa_cleaned_companies.csv'")


Saving cleaned dataset...
Saved cleaned dataset to 'usa_cleaned_companies.csv'


In [16]:
# Display final dataset information
print("\nFinal dataset information:")
print(df_cleaned.info())

# Display sample of cleaned data
print("\nSample of cleaned data:")
print(df_cleaned.head())

# Display success metrics distribution
print("\nSuccess metrics distribution:")
if 'success_binary' in df_cleaned.columns:
    print(f"Success binary distribution:\n{df_cleaned['success_binary'].value_counts(dropna=False)}")
if 'success_class' in df_cleaned.columns:
    print(f"Success class distribution:\n{df_cleaned['success_class'].value_counts(dropna=False)}")


Final dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51637 entries, 0 to 51636
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   entity_type         51637 non-null  object 
 1   name                51635 non-null  object 
 2   category_code       47324 non-null  object 
 3   status              51637 non-null  object 
 4   founded_at          38625 non-null  object 
 5   country_code        51637 non-null  object 
 6   state_code          50662 non-null  object 
 7   city                50184 non-null  object 
 8   region              51637 non-null  object 
 9   first_funding_at    19633 non-null  object 
 10  last_funding_at     19633 non-null  object 
 11  funding_rounds      19732 non-null  float64
 12  funding_total_usd   18142 non-null  float64
 13  first_milestone_at  26670 non-null  object 
 14  last_milestone_at   26670 non-null  object 
 15  milestones          26670

In [18]:
pd.read_csv('usa_cleaned_companies.csv').head(10)

,entity_type,name,category_code,status,founded_at,country_code,state_code,city,region,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_at,updated_at
0,Company,Wetpaint,web,operating,2005-10-17,USA,WA,Seattle,Seattle,2005-10-01,2008-05-19,3.00,39750000.00,2010-09-05,2013-09-18,5.00,17.00,2007-05-25 06:51:27,2013-04-13 03:29:00
1,Company,Flektor,games_video,acquired,NaN,USA,CA,Culver City,Los Angeles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,2007-05-31 21:11:51,2008-05-23 23:23:14
2,Company,There,games_video,acquired,NaN,USA,CA,San Mateo,SF Bay,NaN,NaN,NaN,NaN,2003-02-01,2011-09-23,4.00,12.00,2007-08-06 23:52:45,2013-11-04 02:09:48
3,Company,Thomas Publishing,advertising,operating,NaN,USA,NY,New York,New York,NaN,NaN,NaN,NaN,1999-06-01,1999-06-01,1.00,2.00,2008-08-24 20:21:21,2009-11-19 17:21:00
4,Company,dimension5 labs,advertising,operating,2008-08-01,USA,NM,Santa Fe,Santa Fe,NaN,NaN,NaN,NaN,2008-08-22,2008-08-22,1.00,2.00,2008-08-24 21:54:55,2008-12-21 17:21:53
5,Company,FriendFeed,web,acquired,2007-10-01,USA,CA,Mountain View,SF Bay,2008-02-26,2008-02-26,1.00,5000000.00,2008-05-01,2012-09-13,3.00,14.00,2007-10-01 10:17:13,2013-03-13 21:44:15
6,Company,PoetryVisualized.com,games_video,operating,2008-01-01,USA,CA,Julian,San Diego,NaN,NaN,NaN,NaN,2008-01-01,2008-01-01,1.00,3.00,2008-08-24 22:21:46,2009-04-03 20:32:21
7,Company,Mobclix,mobile,acquired,2008-03-01,USA,CA,Palo Alto,SF Bay,2008-09-01,2008-09-01,1.00,NaN,1995-03-01,2012-09-18,4.00,9.00,2008-08-25 01:32:43,2012-09-22 03:36:21
8,Company,Fitbit,health,operating,2007-10-01,USA,CA,San Francisco,SF Bay,2008-10-10,2013-08-13,5.00,68069200.00,NaN,NaN,NaN,14.00,2008-08-25 02:16:54,2013-12-04 09:52:42
9,Company,MTPV,cleantech,operating,2003-01-01,USA,TX,Austin,Austin,2011-03-08,2012-01-26,3.00,10125293.00,2010-01-01,2010-01-01,1.00,6.00,2011-09-07 02:01:00,2013-04-10 04:35:04
